create spark context

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import pyspark.sql.functions as F
from pyspark.sql.functions import *
spark = SparkSession.builder.getOrCreate()

In [2]:
spark

1. specify the schema for the crime data set

In [6]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, FloatType, TimestampType

In [7]:
structureSchema = StructType([ \
    StructField("X",FloatType(),True), \
    StructField("Y",FloatType(),True), \
    StructField("RowID",StringType(),True), \
    StructField("CrimeDateTime", TimestampType(), True), \
    StructField("CrimeCode", StringType(), True), \
    StructField("Location", StringType(), True),  \
    StructField("Description", StringType(), True),  \
    StructField("Inside_Outside", StringType(), True),  \
    StructField("Weapon", StringType(), True),  \
    StructField("Post", StringType(), True),  \
    StructField("District", StringType(), True),  \
    StructField("Neighborhood", StringType(), True),  \
    StructField("Latitude", FloatType(), True),  \
    StructField("Longitude", FloatType(), True),  \
    StructField('GeoLocation', StringType(), True),  \
    StructField("Premise", StringType(), True),  \
    StructField("VRIName", StringType(), True),  \
    StructField("Total_Incidents", StringType(), True),  \
    StructField("Shape", StringType(), True),  \
  ])

In [8]:
spark.version

'3.2.1'

2. Read the file using the schema definition

In [9]:
#df = spark.read.csv("Part1_Crime_data.csv", header=True,  schema=structureSchema) #nullValue='Null',

In [10]:
df_=(spark.read.format("csv")
    .option("header","true")
    .option("timestampFormat", "yyyy/MM/dd HH:mm:ss+00")
    .schema(structureSchema)
    .option("inferSchema", "true")
    .load("Part1_Crime_data.csv"))

3. Cache the DataFrame

In [11]:
df=df_.persist()

4. Show the count of the rows

In [12]:
row = df.count()
row

513158

5. print the schema

In [13]:
df.printSchema()

root
 |-- X: float (nullable = true)
 |-- Y: float (nullable = true)
 |-- RowID: string (nullable = true)
 |-- CrimeDateTime: timestamp (nullable = true)
 |-- CrimeCode: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Inside_Outside: string (nullable = true)
 |-- Weapon: string (nullable = true)
 |-- Post: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Latitude: float (nullable = true)
 |-- Longitude: float (nullable = true)
 |-- GeoLocation: string (nullable = true)
 |-- Premise: string (nullable = true)
 |-- VRIName: string (nullable = true)
 |-- Total_Incidents: string (nullable = true)
 |-- Shape: string (nullable = true)



6. display first 5 rows

In [14]:
df.show(5)

+---------+---------+-----+-------------------+---------+-------------------+--------------+--------------+-------+----+---------+----------------+--------+---------+------------------+---------------+------------+---------------+-----+
|        X|        Y|RowID|      CrimeDateTime|CrimeCode|           Location|   Description|Inside_Outside| Weapon|Post| District|    Neighborhood|Latitude|Longitude|       GeoLocation|        Premise|     VRIName|Total_Incidents|Shape|
+---------+---------+-----+-------------------+---------+-------------------+--------------+--------------+-------+----+---------+----------------+--------+---------+------------------+---------------+------------+---------------+-----+
|1420074.1| 594160.6|    1|2022-02-26 04:00:00|       4E|  200 W MONUMENT ST|COMMON ASSAULT|          null|     NA| 124|  CENTRAL|    MOUNT VERNON| 39.2975| -76.6193|(39.2975,-76.6193)|           null|        null|              1| null|
|1411374.2| 589791.4|    2|2022-02-26 01:26:30|     

Answer following questions using PySpark
1. What are distinct crime codes?

In [15]:
df.select("CrimeCode").distinct().show()

#total number of crimecode
df.agg(F.countDistinct("CrimeCode")).show()
  
# alternatively, if want to show all the row
# df.select("CrimeCode").distinct().show(df.count(), False) 
# show total number of crimecode
#df.select("CrimeCode").distinct().count()

+---------+
|CrimeCode|
+---------+
|       3P|
|       3K|
|      3BJ|
|       1A|
|       3M|
|       5F|
|       4B|
|       3B|
|       7A|
|      3NF|
|      3EF|
|       3N|
|       5D|
|       6K|
|      3LO|
|      3AF|
|       7B|
|      3GO|
|     3AJF|
|      8GV|
+---------+
only showing top 20 rows

+----------------+
|count(CrimeCode)|
+----------------+
|              85|
+----------------+



2. Count the number of crimes by the crime codes and order by the resulting counts in descending order

In [16]:
df.groupBy("CrimeCode").count().sort(desc("count")).show()

+---------+-----+
|CrimeCode|count|
+---------+-----+
|       4E|91650|
|       6D|68427|
|       5A|43928|
|       7A|40274|
|       6J|27636|
|       6G|26858|
|       6E|24300|
|       6C|23227|
|       4C|22438|
|       5D|14971|
|      3AF|14739|
|       4B|14460|
|       4A|13226|
|       3B|10737|
|       4D| 7232|
|       5B| 6475|
|       9S| 5443|
|       6F| 5081|
|       5C| 4917|
|       6B| 4106|
+---------+-----+
only showing top 20 rows



3. Which neighborhood had most crimes?

In [17]:
df.groupBy("Neighborhood").count().sort(desc("count")).show(1)

+------------+-----+
|Neighborhood|count|
+------------+-----+
|    DOWNTOWN|17799|
+------------+-----+
only showing top 1 row



4. Which month of the year had most crimes?

In [18]:
df_year=df.withColumn("crimes_month", month(df.CrimeDateTime))
df_year.groupBy("crimes_month").count().sort(desc("count")).show(1)

+------------+-----+
|crimes_month|count|
+------------+-----+
|           8|46327|
+------------+-----+
only showing top 1 row



5. What weapons were used?

In [19]:
df_flt=df.filter((df.Weapon.isNotNull()) & (df.Weapon != "NA")).cache()
df_flt.select("Weapon").distinct().show()

+-------+
| Weapon|
+-------+
|  HANDS|
|  KNIFE|
|  OTHER|
|   FIRE|
|FIREARM|
+-------+



6. Which weapon was used the most? 

In [20]:
df_flt.groupby("Weapon").count().sort(desc("count")).show(1)

+-------+-----+
| Weapon|count|
+-------+-----+
|FIREARM|46139|
+-------+-----+
only showing top 1 row

